# Recomendador Spotify - SPRINT I

## Parte 1 - Extracción de Datos

### ● Navegar la documentación de la API de Spotify y explorar los endpoints.

La API de Spotify permite a los desarrolladores acceder a información detallada sobre canciones, artistas, álbumes, playlists, y más. Ofrece una gran variedad de endpoints que permiten extraer datos útiles para análisis de música, creación de recomendadores y otras aplicaciones musicales.

Para interactuar con la API, se requiere:

Autenticación: Se utiliza un OAuth token que puede obtenerse registrando una aplicación en el Spotify for Developers Dashboard.

Rate Limits: Spotify impone límites en la cantidad de solicitudes que se pueden realizar por minuto.



Para obtener el token lo que haremos es usar el client_id y client_secret y codificarlos en base64

La codificación en Base64 es una técnica comúnmente utilizada en el manejo de datos binarios y en la transmisión de información a través de medios que manejan texto, como HTTP. En el contexto de la autenticación con la API de Spotify, se utiliza para codificar las credenciales del cliente (es decir, el client_id y el client_secret) cuando se solicita un token de acceso.

In [4]:
import requests
import base64

# Credenciales de Spoti
client_id = "d0e4bbec0dbf4571aaeeefb235c6c431"
client_secret = "b0bf3147ae79403f86cdc03b78b77497"

# URL para la autenticación
auth_url = 'https://accounts.spotify.com/api/token'

# Codifica el client_id y client_secret en base64
client_credentials = f"{client_id}:{client_secret}"
client_credentials_base64 = base64.b64encode(client_credentials.encode()).decode()

# Cabeceras de la petición
headers = {
    "Authorization": f"Basic {client_credentials_base64}",
}

# Datos del cuerpo de la petición (para obtener token con 'client_credentials')
data = {
    "grant_type": "client_credentials"
}

# Hacer la petición POST para obtener el token
response = requests.post(auth_url, headers=headers, data=data)

# Verificar si la petición fue exitosa
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print("Token de acceso obtenido:", access_token)
else:
    print(f"Error al obtener token: {response.status_code}")
    print(response.text)


Token de acceso obtenido: BQC1DnkhfJo0NaGcdrpzE9K5N2fxYFw88MiakZOJIpYxXzM1gmhh-9kc1xbiO95kzfe65ZfxQubTSITgBlcy2SyV1rfdRYubcP_1A1VP9YeagmuFydo


In [6]:
access_token = 'BQC1DnkhfJo0NaGcdrpzE9K5N2fxYFw88MiakZOJIpYxXzM1gmhh-9kc1xbiO95kzfe65ZfxQubTSITgBlcy2SyV1rfdRYubcP_1A1VP9YeagmuFydo'

Una vez hayamos obtenido el token lo primero que haremos es desarrollar una script para obtener la informacion de una canción

Antes de nada: Para obtener la ID de una canción lo que debemos hacer es ir a la canción que queramos y darle a los 3 puntos y copiar el enlace que nos pone, por ejemplo:
https://open.spotify.com/intl-es/track/6mnfGyPzbzSe8olYGM44pU?si=6e3cf8d12af84d38

Y para obtener la id que necesitamos lo que haremos es quedarne con el código que hay despues de /track/ y antes de ?si= es decir, quedaría así: 
6mnfGyPzbzSe8olYGM44pU


### Y con este script obtendremos lo que sería la informacion básica de una cancion solamente

In [8]:
import requests




# ID de la canción que quieres consultar (puedes obtener esto desde la app de Spotify)
track_id = '6mnfGyPzbzSe8olYGM44pU'  # Ejemplo: canción Luna llena - Beny Jr

# URL del endpoint para obtener información de la canción
track_url = f'https://api.spotify.com/v1/tracks/{track_id}'

# Cabeceras de la petición (incluye el token de acceso)
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Hacer la petición GET para obtener la información de la canción
response = requests.get(track_url, headers=headers)

# Verificar si la petición fue exitosa
if response.status_code == 200:
    track_info = response.json()
    print("Información de la canción:")
    print(f"Nombre: {track_info['name']}")
    print(f"Artista(s): {[artist['name'] for artist in track_info['artists']]}")
    print(f"Álbum: {track_info['album']['name']}")
    print(f"Popularidad: {track_info['popularity']}")
    print(f"Duración (ms): {track_info['duration_ms']}")
else:
    print(f"Error al obtener la canción: {response.status_code}")
    print(response.text)


Información de la canción:
Nombre: Luna llena
Artista(s): ['Beny Jr']
Álbum: Anti Social Cool Kid
Popularidad: 77
Duración (ms): 169846


### Ahora lo que queremos hacer es obtener la información de una playlist y que nos saque la información de cada canción de esa playlist

##### Para ello tendremos que usar el mismo método de la ID, pero esta vez de la playlist, que se obtiene igual que hemos comentado anteriormente


##### Y con este script obtendremos lo que sería la informacion básica de las canciones de la playlist que hayamos elegido


In [148]:
import requests

# ID de la playlist que quieres consultar (puedes obtener esto desde la app de Spotify)
playlist_id = '37i9dQZF1DWUH2AzNQzWua'  # Ejemplo: playlist "Top 50 - Spain"

# URL del endpoint para obtener la información de la playlist
playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}'

# Cabeceras de la petición (incluye el token de acceso)
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Hacer la petición GET para obtener la información de la playlist
response = requests.get(playlist_url, headers=headers)

# Verificar si la petición fue exitosa
if response.status_code == 200:
    playlist_info = response.json()

    # Imprimir algunos detalles básicos de la playlist
    print(f"Nombre de la playlist: {playlist_info['name']}")
    print(f"Descripción: {playlist_info['description']}")
    print(f"Número de canciones: {playlist_info['tracks']['total']}")

    # Obtener la lista de canciones (tracks)
    tracks = playlist_info['tracks']['items']
    
    # Recorrer las canciones y extraer la información
    print("\nCanciones en la playlist:")
    for i, track_item in enumerate(tracks, start=1):
        track = track_item['track']
        track_name = track['name']
        track_artists = ', '.join([artist['name'] for artist in track['artists']])
        track_duration = track['duration_ms'] / 1000  # Convertir duración de ms a segundos
        
        print(f"{i}. {track_name} - {track_artists} ({track_duration} segundos)")
else:
    print(f"Error al obtener la playlist: {response.status_code}")
    print(response.text)


Nombre de la playlist: Acoustic Hits: Oldies but Goodies
Descripción: Instant classics from over the years loaded into one sweet mix.
Número de canciones: 50

Canciones en la playlist:
1. Yellow - Coldplay (266.773 segundos)
2. Iris - The Goo Goo Dolls (289.533 segundos)
3. Somewhere Only We Know - Keane (237.146 segundos)
4. Under the Bridge - Red Hot Chili Peppers (264.306 segundos)
5. How to Save a Life - The Fray (262.533 segundos)
6. Fast Car - Tracy Chapman (296.8 segundos)
7. Use Somebody - Isaac Hale (188.437 segundos)
8. Drive - Incubus (232.453 segundos)
9. Wake Me up When September Ends - Green Day (285.653 segundos)
10. Best Day Of My Life - American Authors (194.24 segundos)
11. Snow (Hey Oh) - Red Hot Chili Peppers (334.666 segundos)
12. FourFiveSeconds - Rihanna, Kanye West, Paul McCartney (188.238 segundos)
13. Hey There Delilah - Plain White T's (232.533 segundos)
14. Slow Dancing in a Burning Room - John Mayer (242.0 segundos)
15. I'm Yours - Jason Mraz (242.946 segun

## Y ahora el objetivo que tenemos es crear un DataFrame con la informacion que creemos que puede ser de utilidad 

In [10]:
import requests
import pandas as pd

# ID de la playlist que quieres consultar 
playlist_id = '37i9dQZF1DWUH2AzNQzWua' 

# URL del endpoint para obtener la información de la playlist
playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}'
audio_features_url = 'https://api.spotify.com/v1/audio-features/'

# Cabeceras de la petición (incluye el token de acceso)
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Hacer la petición GET para obtener la información de la playlist
response = requests.get(playlist_url, headers=headers)

# Verificar si la petición fue exitosa
if response.status_code == 200:
    playlist_info = response.json()

    # Obtener la lista de canciones (tracks)
    tracks = playlist_info['tracks']['items']

    # Crear una lista para almacenar los datos de las canciones
    playlist_data = []

    # Recorrer las canciones y extraer la información
    for track_item in tracks:
        track = track_item['track']
        track_id = track['id']
        track_name = track['name']
        track_artists = ', '.join([artist['name'] for artist in track['artists']])
        track_duration = track['duration_ms'] / 1000  # Convertir duración de ms a segundos
        track_popularity = track['popularity']
        track_explicit = track['explicit']
        track_release_date = track['album']['release_date']

        # Hacer la petición GET para obtener las características de audio de cada canción
        audio_features_response = requests.get(audio_features_url + track_id, headers=headers)
        
        if audio_features_response.status_code == 200:
            audio_features = audio_features_response.json()
            danceability = audio_features['danceability']
            energy = audio_features['energy']
            valence = audio_features['valence']
            tempo = audio_features['tempo']
            acousticness = audio_features['acousticness']
            instrumentalness = audio_features['instrumentalness']
            speechiness = audio_features['speechiness']
        else:
            # En caso de fallo, usamos valores nulos para esos atributos
            danceability = energy = valence = tempo = acousticness = instrumentalness = speechiness = None

        # Añadir la información de la canción a la lista
        playlist_data.append({
            'Nombre': track_name,
            'Artistas': track_artists,
            'Duración (segundos)': track_duration,
            'Popularidad': track_popularity,
            'Explícito': track_explicit,
            'Fecha de Lanzamiento': track_release_date,
            'Danceability': danceability,
            'Energy': energy,
            'Valence': valence,
            'Tempo': tempo,
            'Acousticness': acousticness,
            'Instrumentalness': instrumentalness,
            'Speechiness': speechiness
        })

    # Crear un DataFrame con los datos
    df = pd.DataFrame(playlist_data)

else:
    print(f"Error al obtener la playlist: {response.status_code}")
    print(response.text)


In [11]:
df

,Nombre,Artistas,Duración (segundos),Popularidad,Explícito,Fecha de Lanzamiento,Danceability,Energy,Valence,Tempo,Acousticness,Instrumentalness,Speechiness
0,Yellow,Coldplay,266.773,88,False,2000-07-10,None,None,None,None,None,None,None
1,Iris,The Goo Goo Dolls,289.533,87,False,1998,None,None,None,None,None,None,None
2,Somewhere Only We Know,Keane,237.146,83,False,2004-05-10,None,None,None,None,None,None,None
3,Under the Bridge,Red Hot Chili Peppers,264.306,80,False,1991-09-24,None,None,None,None,None,None,None
4,How to Save a Life,The Fray,262.533,80,False,2005-09-13,None,None,None,None,None,None,None
5,Fast Car,Tracy Chapman,296.800,78,False,1988-04-05,None,None,None,None,None,None,None
6,Use Somebody,Isaac Hale,188.437,55,False,2022-10-14,None,None,None,None,None,None,None
7,Drive,Incubus,232.453,77,False,1999-10-26,None,None,None,None,None,None,None
8,Wake Me up When September Ends,Green Day,285.653,77,False,2004-09-21,None,None,None,None,None,None,None
9,Best Day Of My Life,American Authors,194.240,77,False,2014-01-01,None,None,None,None,None,None,None


### Ahora lo que haremos es limpiar lo que son los Datos, aunque viendo la calidad de los datos con los que trabaja Spotify, modificaremos pocas cosas

In [140]:
import pandas as pd

# Cargar los datos desde el CSV generado
df = pd.read_csv('playlist_info_con_audio_features.csv')

# A) Eliminar filas con valores nulos en las características de audio
df_cleaned = df.dropna(subset=['Danceability', 'Energy', 'Valence', 'Tempo', 'Acousticness', 'Instrumentalness', 'Speechiness'])

# B) Convertir la columna 'Fecha de Lanzamiento' en tipo datetime
df_cleaned['Fecha de Lanzamiento'] = pd.to_datetime(df_cleaned['Fecha de Lanzamiento'], errors='coerce')

# C) Eliminar duplicados (si existen)
df_cleaned = df_cleaned.drop_duplicates(subset=['Nombre', 'Artistas'], keep='first')




df_cleaned.info()

### Y una vez limpios los guardaremos en un csv 

In [145]:
# Guardar los datos limpios en un nuevo CSV
df_cleaned.to_csv('playlist.csv', index=False)

## Nuestro objetivo ahora sera crear 3 dataframes los cuales tengan relación

#### Lo primero que haremos será crear un df con 1200 playlists que nos diga el nombre de la playlist, la descripción, elñ número de canciones que contiene la playlist y la URL

In [14]:
import requests
import pandas as pd
import time

# Acceso a la API de Spotify (reemplaza 'access_token' por tu token válido)
  # Reemplaza con tu token real
headers = {
    "Authorization": f"Bearer {access_token}"
}

# URL de búsqueda de la API de Spotify
search_url = 'https://api.spotify.com/v1/search'

# Función para buscar playlists con un término específico y almacenar los resultados
def search_playlists(query, headers, limit=50, offset=0):
    params = {
        'q': query,
        'type': 'playlist',
        'limit': limit,
        'offset': offset
    }
    
    # Petición GET a la API de Spotify
    response = requests.get(search_url, headers=headers, params=params)
    
    if response.status_code == 200:
        search_results = response.json()
        playlists = search_results['playlists']['items']
        
        # Extraer y devolver la información de las playlists
        return [{
            'Playlist ID': playlist['id'],
            'Nombre Playlist': playlist['name'],
            'Descripción': playlist['description'],
            'Número de canciones': playlist['tracks']['total'],
            'Url de Spotify': playlist['external_urls']['spotify']
        } for playlist in playlists]
    else:
        print(f"Error en la búsqueda de playlists: {response.status_code}")
        print(response.text)
        return []

# Lista para almacenar los datos de playlists
all_playlists_data = []

# Términos de búsqueda para obtener diversidad en los resultados
search_terms = ["Top", "Hits", "Pop", "Rock", "Mood", "Workout"]  
limit = 50  # Número máximo de playlists por búsqueda
offset = 0  # Empezar desde el primer resultado

# Loop para obtener al menos 2000 playlists
for term in search_terms:
    while len(all_playlists_data) < 1200:
        # Obtener datos de playlists para el término de búsqueda actual
        playlists = search_playlists(term, headers, limit=limit, offset=offset)
        
        # Añadir playlists al listado general, asegurando que sean únicas
        all_playlists_data.extend(playlists)
        
        # Incrementar el offset para la siguiente página de resultados
        offset += limit
        
        # Si no hay más playlists, pasamos al siguiente término de búsqueda
        if not playlists:
            break
        
        # Pausa para no sobrecargar la API
        time.sleep(0.1)
    
    # Restablecer el offset para el siguiente término de búsqueda
    offset = 0

    # Si ya alcanzamos las 2000 playlists, salimos del loop
    if len(all_playlists_data) >= 1200:
        break

# Crear DataFrame con los datos de todas las playlists
playlists1 = pd.DataFrame(all_playlists_data[:1200])  # Limitar a las primeras 2000 playlists

# Mostrar el DataFrame
print("DataFrame de Playlists:")
print(playlists1.head())
print(f"\nTotal de playlists recopiladas: {len(playlists1)}")


Error en la búsqueda de playlists: 400
{
  "error" : {
    "status" : 400,
    "message" : "Bad request."
  }
}
DataFrame de Playlists:
              Playlist ID      Nombre Playlist  \
0  37i9dQZEVXbMDoHDwVN2tF      Top 50 - Global   
1  37i9dQZF1DXcBWIGoYBM5M     Today’s Top Hits   
2  37i9dQZEVXbLRQDuF5jeBp         Top 50 - USA   
3  37i9dQZF1DX1lVhptIYRda          Hot Country   
4  37i9dQZF1Fa1IIVtEpGUcU  Your Top Songs 2023   

                                         Descripción  Número de canciones  \
0  Your daily update of the most played tracks ri...                   50   
1                   The hottest 50. Cover: Lady Gaga                   50   
2  Your daily update of the most played tracks ri...                   50   
3      Today's top country hits. Cover: Cody Johnson                   50   
4  Spotify Wrapped presents the songs you loved t...                    0   

                                      Url de Spotify  
0  https://open.spotify.com/playlist/37i9dQZE

#### Lo guardamos ahora en un csv 

In [15]:
playlists1.to_csv("playlists1.csv", index=False)


### Una vez tengamos el df de las playlist, ahora lo que haremos será crear otro que contenga las ID's de las playlists y las ID's de las canciones de cada playlist 

#### Nuestro objetivo con esto será que gracias a esto podremos ver si una cancion está en una o varias playlist y luego será mas fácil a la hora de extraer la informacion de cada canción

In [16]:
import requests
import pandas as pd
import time

# Función para obtener las canciones de una playlist y sus características de audio
def get_playlist_tracks(playlist_id, headers):
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}'

    # Petición para obtener los datos de la playlist
    response = requests.get(playlist_url, headers=headers)

    if response.status_code == 200:
        playlist_info = response.json()
        
        # Lista para almacenar los datos de cada canción
        tracks_data = []

        # Obtener las canciones de la playlist
        tracks = playlist_info['tracks']['items']
        for track_item in tracks:
            track = track_item['track']

            # Verificar si track es None
            if track is None:
                continue

            track_id = track['id']

            # Detalles de la canción
            tracks_data.append({
                'Canción ID': track_id,
                'Playlist ID': playlist_id
            })

        return tracks_data
    else:
        print(f"Error al obtener la playlist: {response.status_code}")
        print(response.text)
        return []

# Acceso a la API de Spotify (reemplaza 'access_token' por tu token válido)
access_token   # Reemplaza con tu token real
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Obtener los IDs de las playlists desde el DataFrame existente
playlist_ids = playlists1['Playlist ID'].tolist()  # Asegúrate de que df_playlists está definido

# Listas para almacenar los datos de las canciones y sus playlists
all_tracks_playlist_data = []

# Recorrer cada playlist y extraer los datos de las canciones
for playlist_id in playlist_ids:
    tracks_data = get_playlist_tracks(playlist_id, headers)
    all_tracks_playlist_data.extend(tracks_data)

    # Pausa para evitar sobrecargar la API
    time.sleep(0.1)

# Crear DataFrame con los datos de las canciones y sus playlists
tracks_playlists1 = pd.DataFrame(all_tracks_playlist_data)

# Mostrar el DataFrame
print("DataFrame de Canciones y Playlists:")
print(tracks_playlists1.head())
print(f"\nTotal de registros en el DataFrame: {len(tracks_playlists1)}")


DataFrame de Canciones y Playlists:
               Canción ID             Playlist ID
0  2plbrEY59IikOBgBGLjaoe  37i9dQZEVXbMDoHDwVN2tF
1  5vNRhkKd0yEAg8suGBpjeY  37i9dQZEVXbMDoHDwVN2tF
2  6dOtVTDdiauQNBQEDOtlAB  37i9dQZEVXbMDoHDwVN2tF
3  7tI8dRuH2Yc6RuoTjxo4dU  37i9dQZEVXbMDoHDwVN2tF
4  1cOboCuWYI2osTOfolMRS6  37i9dQZEVXbMDoHDwVN2tF

Total de registros en el DataFrame: 91438


In [18]:
# Obtener valores únicos de la columna 'Playlist ID'
unique_playlist_ids = tracks_playlists1['Playlist ID'].unique()

# Obtener valores únicos de la columna 'Canción ID'
unique_track_ids = tracks_playlists1['Canción ID'].unique()

# Contar la cantidad de valores únicos
num_unique_playlists = len(unique_playlist_ids)
num_unique_tracks = len(unique_track_ids)

print(f"\nNúmero de Playlist IDs únicos: {num_unique_playlists}")
print(f"Número de Canción IDs únicos: {num_unique_tracks}")




Número de Playlist IDs únicos: 1153
Número de Canción IDs únicos: 55872


In [151]:
tracks_playlists1

,Canción ID,Playlist ID
0,2plbrEY59IikOBgBGLjaoe,37i9dQZEVXbMDoHDwVN2tF
1,5vNRhkKd0yEAg8suGBpjeY,37i9dQZEVXbMDoHDwVN2tF
2,6dOtVTDdiauQNBQEDOtlAB,37i9dQZEVXbMDoHDwVN2tF
3,7tI8dRuH2Yc6RuoTjxo4dU,37i9dQZEVXbMDoHDwVN2tF
4,1cOboCuWYI2osTOfolMRS6,37i9dQZEVXbMDoHDwVN2tF
...,...,...
91433,1WG3EivQuvS70RPnW8D6Fi,3IyNJEsknaSFoUIn8qf1Lr
91434,0KHDthR0XvPoeJaqBc7Sg5,3IyNJEsknaSFoUIn8qf1Lr
91435,5LhI8m7n8xlQmNe3b7Rwis,3IyNJEsknaSFoUIn8qf1Lr
91436,1xgWArLVFD1AKsLzftZYc2,3IyNJEsknaSFoUIn8qf1Lr


#### Lo guardaremos en un csv

In [22]:
tracks_playlists1.to_csv("tracks_playlists.csv", index=False)


## Codigo para extraer la informacion de las canciones del df de ID's usando .unique() 

#### Antes que nada eliminaremos las ID de las canciones que sean None

In [ ]:
tracks_playlists1 = tracks_playlists1.dropna()

In [136]:
import requests
import pandas as pd
import time
import base64
from datetime import datetime

# Función para obtener el token de acceso
def obtener_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()}"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print(f"Error al obtener token: {response.status_code}")
        return None

# Función para obtener datos de canciones en un lote
def obtener_datos_cancion(batch, access_token):
    # URL de la API de Spotify para obtener datos de múltiples canciones
    url_tracks = f"https://api.spotify.com/v1/tracks"
    url_audio_features = f"https://api.spotify.com/v1/audio-features"
    
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Convertir lista de IDs en un solo string separado por comas
    track_ids_str = ",".join(batch)

    # Obtener datos de pistas
    response_tracks = requests.get(f"{url_tracks}?ids={track_ids_str}", headers=headers)
    response_audio_features = requests.get(f"{url_audio_features}?ids={track_ids_str}", headers=headers)

    # Comprobar si las solicitudes fueron exitosas
    if response_tracks.status_code == 200 and response_audio_features.status_code == 200:
        tracks = response_tracks.json()["tracks"]
        audio_features_list = response_audio_features.json()["audio_features"]

        datos_canciones = []
        for track, audio_features in zip(tracks, audio_features_list):
            if track and audio_features:
                datos_cancion = {
                    "Canción ID": track["id"],
                    "Nombre": track["name"],
                    "Artistas": ", ".join([artist["name"] for artist in track["artists"] if artist["name"] is not None]),
                    "Duración (segundos)": track["duration_ms"] / 1000,
                    "Popularidad": track["popularity"],
                    "Explícito": track["explicit"],
                    "Fecha de Lanzamiento": track["album"]["release_date"],
                    "Url de Spotify": track["external_urls"]["spotify"],
                    "Imagen": track["album"]["images"][0]["url"] if track["album"]["images"] else None,
                    "Danceability": audio_features.get("danceability"),
                    "Energy": audio_features.get("energy"),
                    "Valence": audio_features.get("valence"),
                    "Tempo": audio_features.get("tempo"),
                    "Acousticness": audio_features.get("acousticness"),
                    "Instrumentalness": audio_features.get("instrumentalness"),
                    "Speechiness": audio_features.get("speechiness"),
                    "Clave (Key)": audio_features.get("key"),
                    "Modo (Mode)": audio_features.get("mode")
                }
                datos_canciones.append(datos_cancion)
        return datos_canciones
    elif response_tracks.status_code == 429 or response_audio_features.status_code == 429:
        retry_after = int(response_tracks.headers.get("Retry-After", 2))
        print(f"Límite de velocidad alcanzado. Esperando {retry_after} segundos...")
        time.sleep(retry_after)
        return obtener_datos_cancion(batch, access_token)
    else:
        print(f"Error al obtener datos para el lote: {response_tracks.status_code}, {response_audio_features.status_code}")
        return None

# Lista de IDs de canciones
track_ids = tracks_playlists1["Canción ID"].unique()

# Inicializar DataFrame y temporizador
df_canciones = pd.DataFrame(columns=[
    "Canción ID", "Nombre", "Artistas", "Duración (segundos)", "Popularidad", "Explícito", "Fecha de Lanzamiento", 
    "Url de Spotify", "Imagen", "Danceability", "Energy", "Valence", "Tempo", "Acousticness", 
    "Instrumentalness", "Speechiness", "Clave (Key)", "Modo (Mode)"
])

client_id = "10d944ee7fad41d9aa6b0e04a09a96c4"
client_secret = "d5db29896c074e95b7ae73777feadb79"
access_token = obtener_token(client_id, client_secret)

if access_token:
    for i in range(0, len(track_ids), 50):  # Procesar en lotes de 50 IDs
        batch = track_ids[i:i + 50]  # Extraer un lote de hasta 50 IDs
        datos = obtener_datos_cancion(batch, access_token)

        if datos:
            df_canciones = pd.concat([df_canciones, pd.DataFrame(datos)], ignore_index=True)
            df_canciones.to_csv("datos_canciones.csv", index=False)

        # Pausa para evitar el límite de velocidad
        time.sleep(1.0)  # 1 segundo para evitar problemas con el límite de velocidad
    
    print(df_canciones)
else:
    print("No se pudo obtener el token de acceso.")


C:\Users\adria\AppData\Local\Temp\ipykernel_8064\3181853849.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_canciones = pd.concat([df_canciones, pd.DataFrame(datos)], ignore_index=True)


                   Canción ID                       Nombre  \
0      2plbrEY59IikOBgBGLjaoe             Die With A Smile   
1      5vNRhkKd0yEAg8suGBpjeY                         APT.   
2      6dOtVTDdiauQNBQEDOtlAB           BIRDS OF A FEATHER   
3      7tI8dRuH2Yc6RuoTjxo4dU                          Who   
4      1cOboCuWYI2osTOfolMRS6                      Tu Boda   
...                       ...                          ...   
55821  6Yard5LK4aXEn0kBKtHSob  Se Fue La Pantera (En Vivo)   
55822  2fQrrBtuNVMWOHqLyk4EfO               Yo Todo Lo Doy   
55823  0QT0CmOXJCkDpJp6QsBM6H                         Dile   
55824  1WG3EivQuvS70RPnW8D6Fi         Músico, Poeta Y Loco   
55825  0KHDthR0XvPoeJaqBc7Sg5            Ya Te Perdí La Fe   

                                            Artistas  Duración (segundos)  \
0                              Lady Gaga, Bruno Mars              251.667   
1                                   ROSÉ, Bruno Mars              169.917   
2                       

In [147]:
df_canciones


,Canción ID,Nombre,Artistas,Duración (segundos),Popularidad,Explícito,Fecha de Lanzamiento,Url de Spotify,Imagen,Danceability,Energy,Valence,Tempo,Acousticness,Instrumentalness,Speechiness,Clave (Key),Modo (Mode)
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",251.667,100,False,2024-08-16,https://open.spotify.com/track/2plbrEY59IikOBg...,https://i.scdn.co/image/ab67616d0000b27382ea2e...,0.521,0.592,0.535,157.969,0.30800,0.000000,0.0304,6,0
1,5vNRhkKd0yEAg8suGBpjeY,APT.,"ROSÉ, Bruno Mars",169.917,97,False,2024-10-18,https://open.spotify.com/track/5vNRhkKd0yEAg8s...,https://i.scdn.co/image/ab67616d0000b273f8c829...,0.777,0.783,0.939,149.027,0.02830,0.000000,0.2600,0,0
2,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,210.373,97,False,2024-05-17,https://open.spotify.com/track/6dOtVTDdiauQNBQ...,https://i.scdn.co/image/ab67616d0000b27371d62e...,0.747,0.507,0.438,104.978,0.20000,0.060800,0.0358,2,1
3,7tI8dRuH2Yc6RuoTjxo4dU,Who,Jimin,170.887,92,False,2024-07-19,https://open.spotify.com/track/7tI8dRuH2Yc6Ruo...,https://i.scdn.co/image/ab67616d0000b273f02c45...,0.660,0.756,0.838,116.034,0.00289,0.000000,0.0320,0,0
4,1cOboCuWYI2osTOfolMRS6,Tu Boda,"Oscar Maydon, Fuerza Regida",225.880,92,True,2024-09-26,https://open.spotify.com/track/1cOboCuWYI2osTO...,https://i.scdn.co/image/ab67616d0000b273b51d9a...,0.694,0.651,0.471,112.648,0.49600,0.000014,0.0784,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55821,6Yard5LK4aXEn0kBKtHSob,Se Fue La Pantera (En Vivo),"Grupo Firme, Grupo Recluta",202.021,15,False,2021-08-12,https://open.spotify.com/track/6Yard5LK4aXEn0k...,https://i.scdn.co/image/ab67616d0000b273ea5b24...,0.557,0.660,0.907,128.179,0.11800,0.000000,0.0768,3,1
55822,2fQrrBtuNVMWOHqLyk4EfO,Yo Todo Lo Doy,Alfredo Olivas,202.453,73,False,2022-09-30,https://open.spotify.com/track/2fQrrBtuNVMWOHq...,https://i.scdn.co/image/ab67616d0000b2734ad368...,0.597,0.725,0.800,189.960,0.30600,0.000161,0.1360,2,0
55823,0QT0CmOXJCkDpJp6QsBM6H,Dile,Joaquin Guiller,175.291,38,False,2024-10-03,https://open.spotify.com/track/0QT0CmOXJCkDpJp...,https://i.scdn.co/image/ab67616d0000b273da1e41...,0.786,0.555,0.721,145.086,0.56400,0.000000,0.0436,2,0
55824,1WG3EivQuvS70RPnW8D6Fi,"Músico, Poeta Y Loco",Elio Vega,250.642,51,False,2024-10-11,https://open.spotify.com/track/1WG3EivQuvS70RP...,https://i.scdn.co/image/ab67616d0000b273c931ab...,0.681,0.409,0.915,107.770,0.55300,0.000032,0.0325,10,1


In [134]:
len(tracks_playlists1["Canción ID"].unique())


55870